In [44]:
%run model_sucrose.ipynb #Getting model with sucrose as carbon source for modifications

In [19]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH
from cobra import Reaction
from cameo.core.manipulation import swap_cofactors
#Pulling relevant code from cameo. 

Trying to run analysis as a defined function for easy swapping between targets. 

In [3]:
import time
def cofactor_optimization(model, reaction='HACD5'):
    # Starting time for code
    start = time.time()
    
    model.solver = "glpk" #Free software package
    
    biomass = model.reactions.BIOMASS_BS_10 #Reactions leading to biomass formation 
    biomass.lower_bound = 0.1 #Possible to change bounds, but does that make sense to do?
    
    demand = getattr(model.reactions, reaction)
    
    model.objective = demand #Definition on what product we are demanding / looking for and setting it as model objective
    
    pyield = product_yield(demand, model.reactions.EX_sucr_e) #Setting the yield of surfactin based on the surcrose used
    swap_opt = CofactorSwapOptimization(model=model, objective_function=pyield, plot=True, cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
    
    result = swap_opt.run(max_evaluations=100, max_size=5) #Running the analysis
    
    print('Total Time: %.1f' %(time.time()-start))
    
    return result

In [4]:
cofactor_optimization(model, reaction='SP_1')

Starting optimization at Tue, 23 Nov 2021 14:50:50


HBox()

Finished after 00:00:19
Total Time: 32.4


,index,targets,fitness
0,0,"(PDH,)",0.492242
1,1,"(GAPD,)",0.492242
2,3,"(G5SD, KARA1)",0.491395
3,4,"(KARA1,)",0.491378
4,7,"(KAS3, IPMD)",0.490775
5,8,"(MDH,)",0.490764
6,10,"(PPND, AKGDH)",0.489971
7,11,"(AKGDH,)",0.489957
8,13,"(OIVD1r, GLXO1)",0.489778
9,14,"(SP_1,)",0.489773


In [5]:
cofactor_optimization(model, reaction='SP_2')

Starting optimization at Tue, 23 Nov 2021 14:51:22


HBox()

Finished after 00:00:19
Total Time: 36.9


,index,targets,fitness
0,0,"(G6PDH2r, HSDy)",0.488393
1,1,"(G6PDH2r, KAS13)",0.488219
2,2,"(G6PDH2r,)",0.488188
3,4,"(PDH,)",0.487500
4,5,"(GAPD,)",0.487500
5,6,"(GND,)",0.487276
6,7,"(AGPR, KARA1)",0.486705
7,8,"(IPMD, OIVD2)",0.486064
8,9,"(PGCD,)",0.485822
9,10,"(KAS11, AKGDH, HISTD)",0.485382


In [6]:
cofactor_optimization(model, reaction='SP_3')

Starting optimization at Tue, 23 Nov 2021 14:51:59


HBox()

Finished after 00:00:18
Total Time: 33.6


,index,targets,fitness
0,0,"(G6PDH2r, AGPR)",0.486932
1,1,"(G6PDH2r, PPND, MTHFR3)",0.486894
2,2,"(G6PDH2r,)",0.486828
3,3,"(GND,)",0.486828
4,5,"(GAPD,)",0.486459
5,6,"(PDH,)",0.486459
6,8,"(KARA1, HISTD)",0.485611
7,9,"(KARA1,)",0.485303
8,10,"(IPMD,)",0.484985
9,11,"(MDH,)",0.484947


In [7]:
cofactor_optimization(model, reaction='SP_4')

Starting optimization at Tue, 23 Nov 2021 14:52:33


HBox()

Finished after 00:00:19
Total Time: 36.2


,index,targets,fitness
0,0,"(G6PDH2r,)",0.488382
1,2,"(PDH,)",0.488205
2,3,"(GAPD,)",0.488205
3,4,"(GND,)",0.487489
4,5,"(KAS3, AGPR, KARA1)",0.487353
5,6,"(KARA1,)",0.487293
6,7,"(DHDPRy, IPMD)",0.486720
7,8,"(HISTD, IPMD)",0.486694
8,9,"(IPMD,)",0.486678
9,10,"(AKGDH,)",0.485811


In [28]:
model.reactions.G6PDH2r

Reaction identifier,G6PDH2r
Name,Glucose 6-phosphate dehydrogenase
Memory address,0x019e7faa9f70
Stoichiometry,"g6p_c + nadp_c --> 6pgl_c + h_c + nadph_c D-Glucose 6-phosphate + Nicotinamide adenine dinucleotide phosphate --> 6-phospho-D-glucono-1,5-lactone + H+ + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,BSU23850
Lower bound,0.0
Upper bound,999999.0


In [46]:
with model:
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)
    swap_cofactors(reaction = model.reactions.G6PDH2r, model=model, swap_pairs=([model.metabolites.nadp_c, model.metabolites.nadph_c], [model.metabolites.nad_c, model.metabolites.nadh_c]), inplace=True)
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)

0.0
0.0


In [38]:
solution

,fluxes,reduced_costs
EX_etha_e,0.0,-2.496739e-01
EX_drib_e,0.0,0.000000e+00
EX_csn_e,0.0,-7.490218e-01
EX_5mtr_e,0.0,2.770137e-17
EX_dtmp_e,0.0,-4.993479e-01
...,...,...
SP_1,0.0,0.000000e+00
SP_2,0.0,0.000000e+00
SP_3,0.0,0.000000e+00
SP_4,0.0,5.551115e-17


In [39]:
model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.0,-2.496739e-01
EX_drib_e,0.0,2.553513e-15
EX_csn_e,0.0,-7.490218e-01
EX_5mtr_e,0.0,6.383782e-16
EX_dtmp_e,0.0,-4.993479e-01
...,...,...
SP_1,0.0,-1.332268e-15
SP_2,0.0,3.330669e-16
SP_3,0.0,1.554312e-15
SP_4,0.0,1.137979e-15


In [23]:
def swap_cofactors(reaction = 'PDH', model=model, swap_pairs=NADH_NADPH, inplace=True):
    if all(reaction.metabolites.get(met, False) for met in swap_pairs[0]):
        new_coefficients = {met: -reaction.metabolites[met] for met in swap_pairs[0]}
        new_coefficients.update({new_met: reaction.metabolites[met] for met, new_met in zip(*swap_pairs)})
    elif all(reaction.metabolites.get(met, False) for met in swap_pairs[1]):
        new_coefficients = {met: -reaction.metabolites[met] for met in swap_pairs[1]}
        new_coefficients.update({new_met: reaction.metabolites[met] for new_met, met in zip(*swap_pairs)})
    else:
        raise ValueError("%s: Invalid swap pairs %s (%s)" % (reaction.id, str(swap_pairs), reaction.reaction))

    def _inplace(rxn, stoichiometry):
        rxn.add_metabolites(stoichiometry, combine=True)

    def _replace(rxn, stoichiometry):
        new_reaction = Reaction(id="%s_swap" % rxn.id, name=rxn.name,lower_bound=rxn.lower_bound, upper_bound=rxn.upper_bound)
        new_reaction.stoichiometry = rxn
        return new_reaction
    if inplace:
        _inplace(reaction, new_coefficients)
        return reaction
    else:
        new_reaction = _replace(reaction, new_coefficients)
        model.add_reactions([new_reaction])
        reaction.knock_out()
        return new_reaction
    

In [24]:
swap_cofactors(reaction = 'PDH', model=model, swap_pairs=NADH_NADPH, inplace=True)

AttributeError: 'str' object has no attribute 'metabolites'

In [40]:
model.reactions.PDH

Reaction identifier,PDH
Name,Pyruvate dehydrogenase
Memory address,0x019e08d091f0
Stoichiometry,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c Coenzyme A + Nicotinamide adenine dinucleotide + Pyruvate --> Acetyl-CoA + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,( BSU14580 and BSU14590 ) and BSU14600 and BSU14610
Lower bound,0.0
Upper bound,999999.0
